In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score

In [2]:
df = pd.read_csv('/workspaces/Fuzzy_Clustering/datasets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
labels = df["Outcome"].values
df.drop("Outcome", axis=1, inplace=True)
dados = df.to_numpy()

In [4]:
labels

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

In [5]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

# %% [markdown]
# ### Atualização dos centroides

# %% [markdown]
# Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:
# 
# ### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

# %%
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

# %% [markdown]
# ### Atualização da matriz de pertinência

# %% [markdown]
# Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:
# 
# ### $u_{ik} = [\sum_{h=1}^c\{\frac{d(x_k,y_i)}{d(x_k,y_h)}\}^\frac{1}{m-1}]^{-1}$
# 
# onde
# 
# $d(x_k,y_i) = \sum_{j=1}^p(x_k^j-y_i^j)^2$

# %%
def atualizacao_matriz_pertinencia(dados, centroides, m):
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

# %% [markdown]
# ### Fuzzy C-Means

# %% [markdown]
# Ações:
# 1. Inicialização da matriz de pertinência
# 2. Atualização dos centroides
# 3. Atualização da matriz de pertinência
# 
# Critérios de parada:
# 1. Número máximo de iterações atingido
# 2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

# %%
def fcm(dados, num_clusters, m=2, max_iter=10**6, erro=1e-9):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

# %% [markdown]
# ### Índice de Rand Ajustado (IRA)

# %%
def indice_ami(labels, predicted_labels):
    return adjusted_mutual_info_score(labels, predicted_labels)

# %% [markdown]
# ### Simulação de Monte Carlo

# %%
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        #print(predicted_labels)
        idx_ami = indice_ami(labels, predicted_labels)
        indices_rand.append(idx_ami)
    mean_ami = np.mean(indices_rand)
    std_ami = np.std(indices_rand)
    return mean_ami, std_ami

# %% [markdown]
# ### Definição de parâmetros e execução do método

# %%
num_clusters = 2
num_trials = 100
media_ami, dp_ami = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM Clustering Results ({num_trials} trials)")
print(f"Mean AMI: {media_ami:.4f}")
print(f"Standard Deviation of AMI: {dp_ami:.4f}")

Monte Carlo FCM Clustering Results (100 trials)
Mean AMI: 0.0326
Standard Deviation of AMI: 0.0000
